# 文件读取

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import faiss
import pandas as pd
import numpy as np
import torch

import os
os.chdir(r"C:\天池比赛")

In [2]:
df = pd.read_csv(r".\A_audio_recognition_dom_4_20.txt", sep='\t')
df_food = pd.read_csv(r'.\商品.txt', sep='\t')
df_query = df[df['dom']==1.0]

In [3]:
#df_query = df_query.head(50)

In [4]:
df.head()

,uuid,text,dom
0,2028ce68-f4c4-47e7-b61c-263a4ba8a605,我想听红月的你的早安是我的晚安,0
1,c374f644-92fc-4aaa-a7bd-661f57c257e4,天猫精灵暂停五分钟之后再开始,0
2,6968138b-204e-4a12-9366-89f004aa4f56,天猫精灵要个家常花菜炒肉片吧,1
3,50005371-9bfd-45fc-9232-2fd269bc7b5f,来碗家常紫菜蛋花汤吧,1
4,03a15e19-9092-4671-b66c-12990f7af090,天猫精灵要碗红枣山药养胃粥吧,1


# 大模型提取关键信息

In [5]:
import os
from openai import OpenAI
#后续尝试批量请求加速时间
def food_feature(sentence):
    message = [
        {'role': 'system', 'content': '你是一个文本分析助手。输入为句子，输出为从句子中提取的菜名，如果没有具体菜名，则从句子中提取有关菜品的描述特征。例如：输入： 天猫精灵来碗红枣桂圆养胃粥要热乎的 输出：红枣桂圆养胃粥  输入：我在减肥，帮我点个不长胖的外卖 输出：低热量健康'},
        {'role': 'user', 'content':sentence}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-max-latest", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [6]:
df_query['food_feature'] = df_query['text'].progress_apply(food_feature)

100%|██████████| 1988/1988 [35:25<00:00,  1.07s/it]  
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\8730497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['food_feature'] = df_query['text'].progress_apply(food_feature)


In [7]:
df_query[df_query['uuid']=='90626df7-59d9-9f87-9913-a8a0da3846b2']

,uuid,text,dom,food_feature


In [8]:
#df_query.to_csv('df_query.csv', index=False)

In [9]:
#df_query = pd.read_csv(r'D:\天池比赛\df_query.csv')

In [10]:
#尝试批量推理     https://bailian.console.aliyun.com/?tab=model#/model-batch

# 特征工程

In [11]:
# 特征工程挖掘更多特征 1.口味（酸、甜、苦、辣）  2.食材（猪肉、牛肉、小麦） 3.烹饪方式（煎、炸、蒸、煮）4.菜系（川菜、粤菜、鲁菜、法餐、日料） 5.热量（低、中、高）6.特点（美味、管饱、健康）

# 召回

In [12]:
# 召回符合文本的特征对应的菜品？
# 自己训练embedding

### bge——>embedding

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
path = "./BAAI"
# 初始化模型和分词器
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)
 


In [14]:
#转换词向量函数
def cls(text): 
    input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**input).last_hidden_state
    cls_embedding = output[:, 0, :]
    cls_embedding = cls_embedding[0]
    return cls_embedding

In [15]:
#df_food['cls'] = df_food['item_name'].progress_apply(cls)

In [16]:
# 存储时将tensor转换为list
#df_food['cls'] = df_food['cls'].apply(lambda x: x.tolist())
#df_food.to_csv(r'./df_food.csv')

In [17]:

# 读取时将list转回tensor
import ast
df_food = pd.read_csv(r'./df_food.csv')
df_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x)))

In [18]:
df_food[df_food['item_name'].str.contains('加一份')]

,Unnamed: 0,item_name,category_name,cate_1_name,cate_2_name,cate_3_name,cls
690,690,再加一份匠心米饭,白米饭,主食,饭,白米饭,"[tensor(1.3038), tensor(0.3568), tensor(0.2217..."
691,691,再加一份棕香米饭,白米饭,主食,饭,白米饭,"[tensor(0.6392), tensor(0.3504), tensor(-0.111..."
949,949,加一份娃娃菜,浇头,其他,浇头,NaN,"[tensor(0.6703), tensor(-0.4709), tensor(-0.60..."
950,950,加一份水果玉米粒炒在饭里,浇头,其他,浇头,NaN,"[tensor(0.3073), tensor(0.6725), tensor(0.6145..."
951,951,加一份油菜炒进去,浇头,其他,浇头,NaN,"[tensor(0.3769), tensor(0.3605), tensor(0.7039..."
952,952,加一份海菜,浇头,其他,浇头,NaN,"[tensor(0.8070), tensor(0.2724), tensor(0.4163..."
953,953,加一份炖土豆,浇头,其他,浇头,NaN,"[tensor(1.0571), tensor(0.1540), tensor(0.8357..."
954,954,加一份炖蛋,浇头,其他,浇头,NaN,"[tensor(0.8546), tensor(0.3171), tensor(0.8797..."
955,955,加一份熟玉米粒一起炒,浇头,其他,浇头,NaN,"[tensor(0.1548), tensor(0.1659), tensor(0.7044..."
956,956,加一份牛心菜,热菜,菜品,热菜,NaN,"[tensor(0.9471), tensor(0.0566), tensor(0.3564..."


In [19]:
df_query['cls'] = df_query['food_feature'].progress_apply(cls)

100%|██████████| 1988/1988 [02:09<00:00, 15.39it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\1331004436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['cls'] = df_query['food_feature'].progress_apply(cls)


### FAISS 存储

In [20]:
# https://blog.csdn.net/weixin_43114209/article/details/141716443

In [21]:
# 把向量堆叠成 numpy 数组（shape: N x dim）
embeddings = np.stack(df_food['cls'].values).astype('float32')
dim = embeddings.shape[1]

# 创建索引（使用 L2 距离）
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print(f"向量数量：{index.ntotal}")


向量数量：11060


In [22]:
def query(food_cls):
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=5)  # D: 距离, I: 索引
    formatted = '选项:'+" ".join([f"{i+1}.{df_food.loc[s,'item_name']}" for i, s in enumerate(I[0])])
    return formatted

In [23]:
#查询
df_query['Query_faiss'] = df_query['cls'].progress_apply(query)

100%|██████████| 1988/1988 [00:04<00:00, 466.01it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\3337504649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['Query_faiss'] = df_query['cls'].progress_apply(query)


In [24]:
## nlp推荐

In [25]:
def nlp_recommend(food_name,food_choice):
    input = '文本:'+food_name+' '+food_choice
    message = [
        {'role': 'system', 'content': '你是一个智能点餐助手。给你一个文本，以及五个可能的菜品选项。你的任务是从这五个选项中选出与文本最匹配的一个，并且只输出选项的菜名。例如：输入： 文本：天猫精灵来份红枣桂圆养胃粥 选项：1.红枣桂圆粥 2.桂圆红枣粥 3.红枣养胃粥 4.红枣桂圆杂粮粥 5.红枣桂圆枸杞粥  输出：红枣桂圆粥'},
        {'role': 'user', 'content':input}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-plus-latest", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [26]:
df_query['nlp_recommend'] = df_query.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)

100%|██████████| 1988/1988 [30:54<00:00,  1.07it/s]  
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\862560796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_recommend'] = df_query.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)


In [27]:
def faiss_1(s):
    start = s.find("1.") + 2
    end = s.find(" ", start)
    result = s[start:end]
    return result  # 输出: 花菜炒肉片

In [28]:
df_query['faiss_1'] = df_query['Query_faiss'].progress_apply(faiss_1)

100%|██████████| 1988/1988 [00:00<00:00, 680397.91it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\4097527485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['faiss_1'] = df_query['Query_faiss'].progress_apply(faiss_1)


## faiss审核确保输出

In [29]:
df_query['nlp_cls'] = df_query['nlp_recommend'].progress_apply(cls)

100%|██████████| 1988/1988 [02:09<00:00, 15.40it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\3903690941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_cls'] = df_query['nlp_recommend'].progress_apply(cls)


In [30]:
def query_post(food_cls):
#返回最相近的菜品
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=1)  # D: 距离, I: 索引
    return df_food.loc[I[0][0],'item_name']

In [31]:
#查询
df_query['nlp_post'] = df_query['nlp_cls'].progress_apply(query_post)

100%|██████████| 1988/1988 [00:04<00:00, 471.66it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\3269939578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_post'] = df_query['nlp_cls'].progress_apply(query_post)


In [32]:
df_query['nlp_post_cls'] = df_query['nlp_post'].progress_apply(cls)

100%|██████████| 1988/1988 [02:09<00:00, 15.38it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\433197166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_post_cls'] = df_query['nlp_post'].progress_apply(cls)


In [33]:
# 最终推荐菜品与提取特征间的相似度
def similarity(cls1,cls2):
    similarity = torch.cosine_similarity(cls1, cls2, dim=0)
    return similarity.item()

In [34]:
df_query['similarity'] = df_query.progress_apply(lambda row: similarity(row['cls'], row['nlp_post_cls']), axis=1)

100%|██████████| 1988/1988 [00:00<00:00, 46496.31it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_9784\2678904413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['similarity'] = df_query.progress_apply(lambda row: similarity(row['cls'], row['nlp_post_cls']), axis=1)


In [35]:
df_query[df_query['similarity']<0.6]

,uuid,text,dom,food_feature,cls,Query_faiss,nlp_recommend,faiss_1,nlp_cls,nlp_post,nlp_post_cls,similarity
60,1ecfbbb8-34c6-423e-b76b-5ff4719204f8,天猫精灵来点软和的,1,软和的菜品,"[tensor(1.4097), tensor(-0.1054), tensor(0.119...",选项:1.加小份青菜 2.多加点青菜 3.加一份青菜 4.另加小份青菜 5.加一份海菜,加一份青菜,加小份青菜,"[tensor(0.9374), tensor(0.1654), tensor(0.4239...",加一份青菜,"[tensor(0.9374), tensor(0.1654), tensor(0.4239...",0.588680
103,3f289b65-f6bd-466e-8b55-0e069d6b7184,适合老年人的菜,1,适合老年人的菜特征：易消化、清淡、营养均衡,"[tensor(0.3098), tensor(0.1077), tensor(0.1834...",选项:1.多加点青菜 2.加一份葱炒在饭里 3.另加小份青菜 4.加小份青菜 5.加一份青菜,多加点青菜,多加点青菜,"[tensor(0.7515), tensor(0.8534), tensor(0.6539...",多加点青菜,"[tensor(0.7515), tensor(0.8534), tensor(0.6539...",0.582621
448,d252ff69-1d3d-403f-95bc-492a41d6e582,天猫精灵来碗软乎的面条,1,软乎的面条,"[tensor(0.6614), tensor(0.2014), tensor(0.2732...",选项:1.不麻不辣可加两种面 2.黄面条汤面 3.清淡财鱼面 4.粗粮细作面 5.西红柿鸡蛋菠菜面,西红柿鸡蛋菠菜面,不麻不辣可加两种面,"[tensor(0.0708), tensor(-0.6039), tensor(0.486...",西红柿鸡蛋菠菜面,"[tensor(0.0708), tensor(-0.6039), tensor(0.486...",0.582300
1030,e1e275e7-5d0b-4bd4-824c-6694cf107a7f,天猫精灵有煮得糯的吗,1,输出：煮得糯的菜品,"[tensor(0.1730), tensor(-0.1526), tensor(0.111...",选项:1.加一份水果玉米粒炒在饭里 2.加一份豆芽炒在主食里 3.加一份玉米粒炒在饭里 4....,加一份熟玉米粒一起炒,加一份水果玉米粒炒在饭里,"[tensor(0.1548), tensor(0.1659), tensor(0.7044...",加一份熟玉米粒一起炒,"[tensor(0.1548), tensor(0.1659), tensor(0.7044...",0.521326
1075,2d0acb0f-366a-4804-873e-4d5c278df9ef,天猫精灵要容易下咽的,1,容易下咽的菜品特征：**软烂、清淡、易消化**,"[tensor(0.8807), tensor(0.0722), tensor(0.0114...",选项:1.多加点青菜 2.加一份葱炒在饭里 3.加一小份白菜 4.加小份青菜 5.加一点香菜,加小份青菜,多加点青菜,"[tensor(1.1044), tensor(0.5862), tensor(0.4752...",加小份青菜,"[tensor(1.1044), tensor(0.5862), tensor(0.4752...",0.555927
1340,e950b606-b6af-4a8a-8a2c-5d7e5c60cbda,天猫精灵有汤水多的吗,1,汤水多的菜品,"[tensor(0.2101), tensor(-0.2740), tensor(0.443...",选项:1.实力推荐紫菜虾皮疙瘩汤 2.汤里另加娃娃菜 3.加个鸡蛋在糁汤 4.汤里加青菜 5...,实力推荐紫菜虾皮疙瘩汤,实力推荐紫菜虾皮疙瘩汤,"[tensor(0.5080), tensor(0.3619), tensor(0.4893...",实力推荐紫菜虾皮疙瘩汤,"[tensor(0.5080), tensor(0.3619), tensor(0.4893...",0.591932
1422,8309c4ae-60de-43df-89af-3f651e347200,天猫精灵推荐个营养的,1,营养的菜品描述特征：富含蛋白质、维生素、矿物质，低脂肪，均衡搭配。例如：清蒸鱼、西兰花炒虾仁...,"[tensor(0.4372), tensor(0.7409), tensor(0.0324...",选项:1.多加点青菜 2.加一份豆芽炒在主食里 3.加一份葱炒在饭里 4.加一份营养豆芽菜炒...,加一份营养豆芽菜炒在饭里,多加点青菜,"[tensor(0.4509), tensor(0.2970), tensor(0.4758...",加一份营养豆芽菜炒在饭里,"[tensor(0.4509), tensor(0.2970), tensor(0.4758...",0.505374
1536,6a346e28-3f75-4b85-8447-eddce1c229fe,推荐点易嚼的,1,易嚼的菜品,"[tensor(1.0384), tensor(-0.7187), tensor(0.741...",选项:1.加小份青菜 2.加一份豆芽炒在主食里 3.多加点青菜 4.加一小份白菜 5.腐皮青...,腐皮青菜或大白菜,加小份青菜,"[tensor(0.2080), tensor(0.2737), tensor(0.7193...",腐皮青菜或大白菜,"[tensor(0.2080), tensor(0.2737), tensor(0.7193...",0.580459
1563,cbadf096-1148-4091-8af1-d9d4a922d0a6,天猫精灵推荐点啥好,1,无具体菜名或菜品描述特征，无法提取相关内容。,"[tensor(0.5034), tensor(0.4241), tensor(0.3114...",选项:1.点菜需点米饭 2.加一份油菜炒进去 3.多加点青菜 4.加一小份白菜 5.加一份海菜,点菜需点米饭,点菜需点米饭,"[tensor(1.3623), tensor(0.2455), tensor(0.4402...",点菜需点米饭,"[tensor(1.3623), tensor(0.2455), tensor(0.4402...",0.545933
1652,3dcb3817-117e-491d-b526-58ca77ebdfd7,天猫精灵有热乎的推荐吗,1,热乎的推荐,"[tensor(0.7514), tensor(-0.3339), tensor(0.529...",选项:1.超省推荐黑米粥 2.推荐皮蛋瘦肉粥 3.热乎的五常大米饭 4.实力推荐紫菜虾皮疙瘩...,热乎的五常大米饭,超省推荐黑米粥,"[tensor(1.2336), tensor(-0.1787), tensor(-0.44...",热乎的五常大米饭,"[tensor(1.2336), tensor(-0.1787), tensor(-0.44...",0.556805


#### 比较不同输出

In [36]:
df_diff = df_query[df_query['nlp_recommend'] != df_query['nlp_post']]


In [37]:
df_diff = df_diff[['text','food_feature','nlp_recommend','nlp_post','Query_faiss']]

In [38]:
df_diff

,text,food_feature,nlp_recommend,nlp_post,Query_faiss
205,来份番茄炒蛋吧,番茄炒蛋,番茄炒蛋,番茄炒鸡蛋,选项:1.番茄炒鸡蛋 2.西红柿炒蛋 3.小番茄炒蛋 4.番茄炒蛋饭 5.西红柿炒蛋炒蛋
606,来碗桂圆莲子杂粮粥吧,桂圆莲子杂粮粥,桂圆莲子杂粮粥,桂圆莲子粥,选项:1.桂圆莲子粥 2.桂圆莲子养生粥 3.桂圆红枣莲子粥 4.桂圆莲子红枣粥 5.红枣桂...
787,天猫精灵要个肉末茄子煲吧,肉末茄子煲,肉末茄子煲,肉末茄子饭,选项:1.肉末茄子饭 2.肉末茄子炒菜 3.肉末炒茄子 4.肉末茄子盖饭 5.肉末茄子盖浇饭
816,天猫精灵来碗暖胃的小米粥吧,小米粥,小米粥,大米小米粥,选项:1.大米小米粥 2.粗粮小米粥 3.早餐小米粥 4.小米粥南瓜粥 5.牛奶小米粥
840,天猫精灵来碗家常乌鸡汤吧,家常乌鸡汤,家常乌鸡汤,营养乌鸡汤,选项:1.营养乌鸡汤 2.滋补乌鸡汤 3.养身乌鸡汤 4.招牌乌鸡汤 5.养颜乌鸡汤
893,天猫精灵要份暖心鸡丝粥吧,暖心鸡丝粥,暖心鸡丝粥,养生鸡丝粥,选项:1.养生鸡丝粥 2.美味鸡丝粥 3.香滑鸡丝粥 4.鸡丝蔬菜粥 5.玉米鸡丝粥
914,天猫精灵来碗暖胃的小米粥吧,小米粥,小米粥,大米小米粥,选项:1.大米小米粥 2.粗粮小米粥 3.早餐小米粥 4.小米粥南瓜粥 5.牛奶小米粥
1014,天猫精灵来碗萝卜肉圆粉丝汤吧,萝卜肉圆粉丝汤,萝卜肉圆粉丝汤,萝卜肉圆汤,选项:1.萝卜肉圆汤 2.萝卜粉丝汤 3.西红柿肉圆粉丝汤 4.萝卜丝粉条汤 5.萝卜羊肉粉丝汤
1054,天猫精灵来碗山药红薯粥养胃的,山药红薯粥,山药红薯粥,山药红豆粥,选项:1.山药红豆粥 2.山药红枣粥 3.红枣山药粥 4.红豆山药粥 5.紫薯山药粥
1206,天猫精灵要个鲜贝手打丸子汤吧,鲜贝手打丸子汤,手打鲜贝丸子汤,外卖手打鲜贝丸子汤,选项:1.外卖手打鲜贝丸子汤 2.手打生汆丸子汤 3.手打鱼丸汤 4.手工丸子汤 5.手打鲅...


# 提交文件格式

In [39]:
df_result = pd.merge(df, df_query[['uuid','nlp_post']], how='left', on='uuid')

In [40]:
df_result.columns

Index(['uuid', 'text', 'dom', 'nlp_post'], dtype='object')

In [41]:

df_result.to_csv(r".\result\Results420.txt", 
          sep='\t', 
          header=None, 
          index=False, 
          na_rep='')